In [1]:
from  sklearn.datasets import make_regression
from matplotlib import pyplot
import pandas as pd    
import numpy as np
from DGen import DGenerator

In [2]:
df_cadre = DGenerator.generate_cadres(3000, 20, 10, 4, 101)

In [3]:
df_variables = DGenerator.generate_variables(df_cadre, 3000, 10, 2, 5, 10)

UnboundLocalError: local variable 'cadre' referenced before assignment